In [1]:
import os
# os.environ["CUDA_HOME"] = CUDA_HOME
from torch.utils.cpp_extension import CUDA_HOME
os.environ["CUDA_HOME"] = CUDA_HOME
os.environ["CUDA_PATH"] = CUDA_HOME



/home/jlin/miniconda3/envs/mmlabdev/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# need to restart the notebook after installing these

%cd Grounded-Segment-Anything 
!pip install -q -r requirements.txt
%cd GroundingDINO
!pip install -q .
%cd ../segment_anything
!pip install -q .
%cd ..

In [ ]:
!wget https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha2/groundingdino_swinb_cogcoor.pth
!wget https://huggingface.co/ybelkada/segment-anything/resolve/main/checkpoints/sam_vit_h_4b8939.pth

In [2]:
import os, sys

sys.path.append(os.path.join(os.getcwd(), "GroundingDINO"))

import argparse
import os
import copy

import numpy as np
import torch
from PIL import Image, ImageDraw, ImageFont
from torchvision.ops import box_convert

# Grounding DINO
import GroundingDINO.groundingdino.datasets.transforms as T
from GroundingDINO.groundingdino.models import build_model
from GroundingDINO.groundingdino.util import box_ops
from GroundingDINO.groundingdino.util.slconfig import SLConfig
from GroundingDINO.groundingdino.util.utils import clean_state_dict, get_phrases_from_posmap
from GroundingDINO.groundingdino.util.inference import annotate, load_image, predict

# import supervision as sv

# segment anything
from segment_anything import build_sam, SamPredictor 
import cv2
import numpy as np
import matplotlib.pyplot as plt


# diffusers
import PIL
import requests
import torch
from io import BytesIO
from diffusers import StableDiffusionInpaintPipeline


from huggingface_hub import hf_hub_download

In [3]:
def load_model_hf(repo_id, filename, ckpt_config_filename, device='cpu'):
    cache_config_file = hf_hub_download(repo_id=repo_id, filename=ckpt_config_filename)

    args = SLConfig.fromfile(cache_config_file) 
    model = build_model(args)
    args.device = device

    # cache_file = hf_hub_download(repo_id=repo_id, filename=filename)
    checkpoint = torch.load('groundingdino_swinb_cogcoor.pth', map_location='cpu')
    log = model.load_state_dict(clean_state_dict(checkpoint['model']), strict=False)
    # print("Model loaded from {} \n => {}".format(cache_file, log))
    _ = model.eval()
    return model   


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

ckpt_repo_id = "ShilongLiu/GroundingDINO"
ckpt_filenmae = "groundingdino_swinb_cogcoor.pth"
ckpt_config_filename = "GroundingDINO_SwinB.cfg.py"
groundingdino_model = load_model_hf(ckpt_repo_id, ckpt_filenmae, ckpt_config_filename, device=device)


sam_checkpoint = 'sam_vit_h_4b8939.pth'
sam = build_sam(checkpoint=sam_checkpoint)
sam.to(device=device)
sam_predictor = SamPredictor(sam)

/home/jlin/miniconda3/envs/mmlabdev/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525539683/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
import pycocotools.mask as maskUtils
from typing import Dict
import os.path as osp
import os
from pathlib import Path
from tqdm import tqdm
import json

def dict2json(adict: dict, json_path: str):
    with open(json_path, "w", encoding="utf-8") as f:
        f.write(json.dumps(adict, ensure_ascii=False))

def json2dict(json_path: str):
    with open(json_path, 'r', encoding='utf8') as f:
        metadata = json.loads(f.read())
    return metadata

def mask2rle(mask: np.ndarray, decode_for_json: bool = True) -> Dict:
    mask_rle = maskUtils.encode(np.array(
                        mask[..., np.newaxis] > 0, order='F',
                        dtype='uint8'))[0]
    if decode_for_json:
        mask_rle['counts'] = mask_rle['counts'].decode()
    return mask_rle

def show_mask(mask, image, random_color=True):
    if isinstance(mask, torch.Tensor):
        mask = mask.cpu().numpy()
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.8])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    
    annotated_frame_pil = Image.fromarray(image).convert("RGBA")
    mask_image_pil = Image.fromarray((mask_image * 255).astype(np.uint8)).convert("RGBA")

    return np.array(Image.alpha_composite(annotated_frame_pil, mask_image_pil))

IMG_EXT = {'.bmp', '.jpg', '.png', '.jpeg'}
def find_all_imgs(img_dir, abs_path=False):
    imglist = []
    dir_list = os.listdir(img_dir)
    for filename in dir_list:
        file_suffix = Path(filename).suffix
        if file_suffix.lower() not in IMG_EXT:
            continue
        if abs_path:
            imglist.append(osp.join(img_dir, filename))
        else:
            imglist.append(filename)
    return imglist

In [9]:
def infer_saveanns(annp, img_dir, savep, box_thr = 0.3, max_preds = 100):

    anns = json2dict(annp)

    det_annotations = []
    obj_id = 0

    for kk, img_meta in enumerate(tqdm(anns['images'])):
        img_id = img_meta['id']
        imgp = osp.join(img_dir, img_meta['file_name'])

        image_source, image = load_image(imgp)

        boxes, logits, phrases = predict(
            model=groundingdino_model, 
            image=image, 
            caption=TEXT_PROMPT, 
            box_threshold=box_thr, 
            text_threshold=TEXT_TRESHOLD
        )
        # if kk > 10:
        #     break

        if boxes.shape[0] == 0:
            continue

        if len(boxes) > max_preds:
            sids = torch.argsort(logits)
            boxes = boxes[sids]
            logits = logits[sids]
            boxes = boxes[:max_preds]
            logits = logits[:max_preds]
        # print(boxes, logits)
        # break
        scores = [score.item() for score in logits]


        annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)
        annotated_frame = annotated_frame[...,::-1] # BGR to RGB

        sam_predictor.set_image(image_source)
        H, W, _ = image_source.shape
        boxes_xyxy = box_ops.box_cxcywh_to_xyxy(boxes) * torch.Tensor([W, H, W, H])

        transformed_boxes = sam_predictor.transform.apply_boxes_torch(boxes_xyxy, image_source.shape[:2]).to(device)
        masks, _, _ = sam_predictor.predict_torch(
                    point_coords = None,
                    point_labels = None,
                    boxes = transformed_boxes,
                    multimask_output = False,
                )
        

        for mask, bbox, score in zip(masks, boxes_xyxy, scores):
            mask = mask.squeeze().cpu().numpy().astype(np.uint8) 
            area = int(mask.sum())
            mask = mask2rle(mask)
            bbox = bbox.squeeze().numpy().astype(np.float32)
            bbox[2:] -= bbox[:2]
            bbox = bbox.tolist()
            
            det_annotations.append({'id': obj_id, 'category_id': 0, 'iscrowd': 0,
                'segmentation': mask, 'image_id': img_id, 'area': area, 'bbox': bbox, 'score': score
            })
            obj_id += 1
        # image_id += 1


    det_meta = {"info": {},"licenses": [], "images": anns['images'], 
                "annotations": det_annotations, "categories": CATEGORIES}
    dict2json(det_meta, savep)
    print(f'rst saved to {savep}')


TEXT_PROMPT = "person"
BOX_TRESHOLD = 0.3
TEXT_TRESHOLD = 0.25

CATEGORIES = [
    {"id": 0, "name": "object", "isthing": 1}
]

In [10]:
ANISEG_DIR = r'/home/jlin/code_repo/AnimeInstanceSegmentation/workspace/annotated_images/val'
ANISEG_ANNS = r'/home/jlin/code_repo/AnimeInstanceSegmentation/workspace/annotated_images/annotations/det_val.json'
SAVEP = r'/home/jlin/code_repo/AnimeInstanceSegmentation/local_benchmark/aniseg_gs_std.json'
infer_saveanns(ANISEG_ANNS, ANISEG_DIR, SAVEP, 0.05, 100)

100%|██████████| 945/945 [10:51<00:00,  1.45it/s]

rst saved to /home/jlin/code_repo/AnimeInstanceSegmentation/local_benchmark/aniseg_gs_std.json


In [11]:
ANIINSSEG_ROOT_DIR = '/mnt/nvme1/dldata/anime_instance_dataset/val'
ANIINSSEG_ROOT_EVAL_ANN = '/mnt/nvme1/dldata/anime_instance_dataset/annotations/det_val.json'
SAVEP = r'/home/jlin/code_repo/AnimeInstanceSegmentation/local_benchmark/aisval_gs_std.json'
infer_saveanns(ANIINSSEG_ROOT_EVAL_ANN, ANIINSSEG_ROOT_DIR, SAVEP, 0.05)

100%|██████████| 7496/7496 [1:33:08<00:00,  1.34it/s]


rst saved to /home/jlin/code_repo/AnimeInstanceSegmentation/local_benchmark/aisval_gs_std.json


In [12]:
MANGA109_ROOT_DIR = '/mnt/nvme1/dldata/Manga109_released_2020_12_18/images'
MANGA109_EVAL_ANN = '/home/jlin/code_repo/AnimeInstanceSegmentation/workspace/manga109coco_val.json'
SAVEP = r'/home/jlin/code_repo/AnimeInstanceSegmentation/local_benchmark/manga109val_gs_std.json'
infer_saveanns(MANGA109_EVAL_ANN, MANGA109_ROOT_DIR, SAVEP, 0.05)

100%|██████████| 1034/1034 [19:36<00:00,  1.14s/it]


rst saved to /home/jlin/code_repo/AnimeInstanceSegmentation/local_benchmark/manga109val_gs_std.json


In [14]:
COMPARE_DIR = '/home/jlin/code_repo/AnimeInstanceSegmentation/workspace/compare'

COMPARE_ANN = '/home/jlin/code_repo/AnimeInstanceSegmentation/workspace/compare/maskrcnn.json'
SAVEP = r'/home/jlin/code_repo/AnimeInstanceSegmentation/workspace/compare/gs.json'
infer_saveanns(COMPARE_ANN, COMPARE_DIR, SAVEP, 0.3)

100%|██████████| 12/12 [00:08<00:00,  1.38it/s]

rst saved to /home/jlin/code_repo/AnimeInstanceSegmentation/workspace/compare/gs.json
